In [30]:
#! /data/zhangxingyu/anaconda3/envs/py38/bin/python
import argparse
import pandas as pd
from tqdm import tqdm
import yaml
import math
import numpy as np
from datetime import datetime
import json
import bisect

config = yaml.load(open('../config/config.yaml'))
exception_name_columns = config['exception_name']['train']

<ipython-input-30-d03e5c9cd20a>:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(open('../config/config.yaml'))


In [2]:
days = 60
nc_exceptions_train = pd.read_csv('../'+config['data']['process']['train']['nc_exceptions_key_ip_column_name_value_timelist'])
# test sample 里所有nc 都不在test exception里面出现，因此不需要nc_exceptions_test的信息。
nc_exceptions_test = pd.read_csv('../'+config['data']['process']['test']['nc_exceptions_key_ip_column_name_value_timelist'])
nc_sample_label = pd.read_csv('../'+config['data']['raw']['test']['nc_sample_label'])

In [3]:
for exception_name in exception_name_columns:
    nc_exceptions_train[exception_name] = nc_exceptions_train[exception_name].fillna('[]').apply(eval)
    nc_exceptions_test[exception_name] = nc_exceptions_test[exception_name].fillna('[]').apply(eval)


In [33]:
nc_sample_label_with_exceptions = nc_sample_label.join(nc_exceptions_train.set_index('nc_ip'), how='left', on='nc_ip').fillna('')
str2date = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f") if '.' in x else datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
series2date = lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S')
date2str = lambda x: x.strftime("%Y-%m-%d %H:%M:%S.%f")

nc_sample_label_with_exceptions['sample_time'] = series2date(nc_sample_label_with_exceptions['sample_time'])
nc_sample_label_with_exceptions['start_time'] = nc_sample_label_with_exceptions['sample_time'] - pd.DateOffset(days=days)


In [40]:
# for exception_name in exception_name_columns:
    # nc_exceptions_train[exception_name] = nc_exceptions_train[exception_name].fillna('[]').apply(eval)
    # nc_exceptions_train[exception_name] = nc_exceptions_train[exception_name].fillna('[]').apply(lambda x: x.strip("[]").split(", "))
    # nc_exceptions_test[exception_name] = nc_exceptions_test[exception_name].fillna('[]').apply(lambda x: x.strip("[]").split(", "))
    # nc_sample_label_with_exceptions[exception_name] = nc_sample_label_with_exceptions[exception_name].fillna('[]').apply(eval)

In [31]:
def get_date_cnt_from_intervals(start_date, end_date, date_list):
    if date_list == '': return 0
    return bisect.bisect_left(date_list, end_date.str) - bisect.bisect_left(date_list, start_date)


In [40]:
for exception_name in tqdm(exception_name_columns, total=len(exception_name_columns)):
    exception_cnt_name = exception_name + '_cnt' 
    nc_sample_label_with_exceptions[exception_cnt_name] = nc_sample_label_with_exceptions.apply(lambda x: get_date_cnt_from_intervals(date2str(x['start_time']), date2str(x['sample_time']), x[exception_name]), axis=1)

100%|██████████| 57/57 [11:25<00:00, 12.02s/it]


In [42]:
nc_sample_label_with_exceptions[['nc_ip', 'sample_time', 'nc_down_label'] + [exception_name + '_cnt' for exception_name in exception_name_columns]].to_csv('../'+config['data']['features']['train']['n_exceptions'], index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/features/ecsmpf_round1_train_20210625/round1_train_n_exceptions.csv'

In [7]:

res_tuples = []
for i in tqdm(nc_sample_label_with_exceptions.index, total=nc_sample_label_with_exceptions.shape[0]):
    row = nc_sample_label_with_exceptions.loc[i]
    # new_tuple = [row['nc_ip'], row['sample_time'], row['nc_down_label']]\
    #     + [len([1 for date_str in row[exception_name] if date_str and row['start_time'] <= str2date(date_str) < row['sample_time']]) for exception_name in exception_name_columns]
    # res_tuples.append(new_tuple)
    pass
res_columns = ['nc_ip', 'sample_time', 'nc_down_label'] + exception_name_columns
res_tbl = pd.DataFrame(res_tuples, columns=res_columns)


 53%|█████▎    | 281770/530922 [00:35<00:31, 8016.84it/s]


KeyboardInterrupt: 

SyntaxError: invalid syntax (<ipython-input-5-bd5c75f13f21>, line 12)

,nc_ip,sample_time,nc_down_label,exception_name_10,exception_name_11,exception_name_12,exception_name_14,exception_name_15,exception_name_16,exception_name_17,...,exception_name_55,exception_name_56,exception_name_57,exception_name_58,exception_name_59,exception_name_6,exception_name_60,exception_name_7,exception_name_8,exception_name_9
0,f4724808-d7ec-4cad-9179-29414b1188b0,2020-12-19 15:25:00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2020-12-19 15:23:08', '2020-12-19 15:23:08',...",NaN,NaN,NaN,NaN,NaN,NaN
1,e5336393-ecca-443d-8df8-f09e928c8f00,2017-09-09 15:25:00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2017-09-09 15:23:00', '2017-09-09 15:23:00',...",NaN,NaN,NaN,NaN,NaN,NaN
2,30d42ae0-76e3-4121-9098-0a3557cc8a59,2016-06-17 15:25:00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2016-06-17 15:22:50', '2016-06-17 15:22:50',...",NaN,NaN,NaN,NaN,NaN,NaN
3,88226889-3382-4f0d-8f4f-600e67c9111f,2018-07-17 15:25:00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2018-07-17 15:22:57', '2018-07-17 15:22:57',...",NaN,NaN,NaN,NaN,NaN,NaN
4,c2e430d3-732f-4c0e-b740-92d01195fba6,2017-11-22 17:55:00,False,['2017-11-23 09:19:29'],NaN,NaN,NaN,NaN,"['2017-11-22 17:52:16.810000', '2017-11-23 09:...","['2017-11-22 17:52:16', '2017-12-08 09:41:27',...",...,NaN,NaN,"['2017-11-22 17:52:16', '2017-11-23 09:19:29',...","['2017-11-20 15:22:54', '2017-11-20 15:22:54',...",NaN,NaN,NaN,NaN,NaN,['2017-11-23 09:19:29']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530917,f55ea661-dc45-4056-bff6-5ab732af8a04,2017-12-17 12:10:00,False,NaN,NaN,NaN,NaN,"['2017-12-08 14:10:19', '2018-01-15 05:28:02',...","['2017-11-09 20:48:54.617000', '2017-11-10 20:...","['2017-11-09 20:48:54', '2017-11-10 20:35:09',...",...,NaN,NaN,"['2017-11-09 20:48:54', '2017-11-10 20:35:09',...","['2017-11-09 20:48:54', '2017-11-09 20:48:54',...",NaN,"['2017-12-08 14:10:19.273000', '2018-01-29 02:...",NaN,NaN,NaN,"['2017-12-07 14:24:04', '2018-01-14 05:41:46',..."
530918,f55ea661-dc45-4056-bff6-5ab732af8a04,2017-12-28 09:40:00,False,NaN,NaN,NaN,NaN,"['2017-12-08 14:10:19', '2018-01-15 05:28:02',...","['2017-11-09 20:48:54.617000', '2017-11-10 20:...","['2017-11-09 20:48:54', '2017-11-10 20:35:09',...",...,NaN,NaN,"['2017-11-09 20:48:54', '2017-11-10 20:35:09',...","['2017-11-09 20:48:54', '2017-11-09 20:48:54',...",NaN,"['2017-12-08 14:10:19.273000', '2018-01-29 02:...",NaN,NaN,NaN,"['2017-12-07 14:24:04', '2018-01-14 05:41:46',..."
530919,f55ea661-dc45-4056-bff6-5ab732af8a04,2018-01-03 08:15:00,False,NaN,NaN,NaN,NaN,"['2017-12-08 14:10:19', '2018-01-15 05:28:02',...","['2017-11-09 20:48:54.617000', '2017-11-10 20:...","['2017-11-09 20:48:54', '2017-11-10 20:35:09',...",...,NaN,NaN,"['2017-11-09 20:48:54', '2017-11-10 20:35:09',...","['2017-11-09 20:48:54', '2017-11-09 20:48:54',...",NaN,"['2017-12-08 14:10:19.273000', '2018-01-29 02:...",NaN,NaN,NaN,"['2017-12-07 14:24:04', '2018-01-14 05:41:46',..."
530920,f55ea661-dc45-4056-bff6-5ab732af8a04,2018-01-18 04:50:00,False,NaN,NaN,NaN,NaN,"['2017-12-08 14:10:19', '2018-01-15 05:28:02',...","['2017-11-09 20:48:54.617000', '2017-11-10 20:...","['2017-11-09 20:48:54', '2017-11-10 20:35:09',...",...,NaN,NaN,"['2017-11-09 20:48:54', '2017-11-10 20:35:09',...","['2017-11-09 20:48:54', '2017-11-09 20:48:54',...",NaN,"['2017-12-08 14:10:19.273000', '2018-01-29 02:...",NaN,NaN,NaN,"['2017-12-07 14:24:04', '2018-01-14 05:41:46',..."
